In [4]:
import re
from pathlib import Path
import os
import pandas as pd
import altair as alt
import numpy as np

In [5]:
csv_file = os.path.join(Path.cwd(), "IS 457 Franchise Dataset_exclusive.xlsx")
df = pd.read_excel(csv_file)

In [6]:
def score_calc(t1, t2, t3):
    if t3 == "pass":
        return 3
    elif t2 == "pass":
        return 2
    elif t1 == "pass":
        return 1
    else:
        return 0

In [7]:
df["score"] = df.apply(lambda x: score_calc(x["Test 1"], x["Test 2"], x["Test 3"]), axis=1)
df = df.sort_values(by=['Franchise name', 'Date of Release'], ignore_index = True)
df["Date of Release"] = df["Date of Release"].str.replace(',', '')
df.head(5)

,Franchise name,Movie name,Date of Release,Test 1,Test 2,Test 3,score
0,Alien,Aliens,July 14 1986,pass,pass,pass,3
1,Alien,Prometheus,June 8 2012,pass,pass,pass,3
2,Alien,Alien: Covenant,May 19 2017,pass,pass,pass,3
3,Alien,Alien 3,May 22 1992,fail,fail,fail,0
4,Alien,Alien,May 25 1979,pass,pass,pass,3


In [8]:
def month_only (date):
    mon =  {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
    date = date.strip()
    if date.isalpha():
        return int(mon[date])
    else:
        for i in date:
            if not(i.isalpha()):
                date = date.replace(i,'')
        return int(mon[date])

In [9]:
def day_only (date):
    day = ""
    cnt = 2
    for i in date:
        if (cnt < 0):
            print(date)
            print(cnt)
        if i.isdigit():
            day += i
            cnt-=1
    return int(day)

In [10]:
df["Year"] = df["Date of Release"].str[-4:]
df["Month"] = df["Date of Release"].str[:-7].apply(month_only)
df["Day"] = df["Date of Release"].str[:-4].apply(day_only)
df["iso"] = pd.to_datetime(df["Year"].apply(str) + '-' + df["Month"].apply(str) + '-' + df["Day"].apply(str))
df.head(5)

,Franchise name,Movie name,Date of Release,Test 1,Test 2,Test 3,score,Year,Month,Day,iso
0,Alien,Aliens,July 14 1986,pass,pass,pass,3,1986,7,14,1986-07-14
1,Alien,Prometheus,June 8 2012,pass,pass,pass,3,2012,6,8,2012-06-08
2,Alien,Alien: Covenant,May 19 2017,pass,pass,pass,3,2017,5,19,2017-05-19
3,Alien,Alien 3,May 22 1992,fail,fail,fail,0,1992,5,22,1992-05-22
4,Alien,Alien,May 25 1979,pass,pass,pass,3,1979,5,25,1979-05-25


In [11]:
df1 = df[["Franchise name","Movie name","iso","score","Year"]]
df1 = df1.rename(columns={"iso":"Date"})
source = df1
df1["emp"] = "a"
#df1["emp"] = pd.to_numeric(df1["emp"])
df1.head()


,Franchise name,Movie name,Date,score,Year,emp
0,Alien,Aliens,1986-07-14,3,1986,a
1,Alien,Prometheus,2012-06-08,3,2012,a
2,Alien,Alien: Covenant,2017-05-19,3,2017,a
3,Alien,Alien 3,1992-05-22,0,1992,a
4,Alien,Alien,1979-05-25,3,1979,a


In [12]:
selector = alt.selection_point(fields=['Franchise name'])

color_scale = alt.Scale(domain=['Alien', 'Fast and furious', 'Harry Potter', 'John Wick', 'MadMax',
       'Mission impossible', 'Pirates of the Caribbean', 'Predator',
       'Star Wars', 'Terminator', 'The Matrix', 'X-Men'],
                        range=['#1FC3AA', '#8624F5', '#0E68C1', '#D49410', '#BE66A6', '#EE3367', '#9A536E', '#DC6A6F', '#4C36C4', '#AAC77A', '#27434B', '#AB4901'])

color = alt.condition(
    selector,
    alt.Color('Franchise name:N', scale=color_scale).legend(None),
    alt.value('lightgray')
)

base = alt.Chart(source).properties(
    width=500,
    height=500
).add_params(selector).transform_filter(
    selector
)

bars = base.mark_bar(size=30).encode(
    x=alt.X("mean(score):Q"),
    y=alt.Y("Franchise name:N").sort('-x'),
    color=alt.condition(
        selector,
        'Franchise name:N',
        alt.value('lightgray'),
        scale=color_scale),
    tooltip=[alt.Tooltip('mean(score):Q', title='Average Bechdel Test Score'), alt.Tooltip('count(Franchise name):Q', title='Number of films in franchise')],
).properties()

yrule = (
    base.mark_rule(strokeDash=[12, 6], size=2).encode(y=alt.datum(df1["score"].mean()))
)

xrule = (
    base.mark_rule(strokeDash=[12, 6], size=2).encode(x=alt.datum(df1["score"].mean()))
)

lines = base.mark_line(point=True).encode(
    x='Date:T',
    y='score:Q',
    color=alt.condition(
        selector,
        'Franchise name:N',
        alt.value('lightgray'),
        scale=color_scale),
    tooltip=[alt.Tooltip('Movie name:N', title='Film: '), alt.Tooltip('Date:T', title='Date released in USA'), alt.Tooltip('score:Q', title='Test Passed')],
).add_params(
    selector
).transform_filter(
    selector
)

dep = lines + yrule
prime = bars + xrule

viz_v1 = alt.hconcat(prime, dep).configure_point(size=120)

viz_v1.properties().save("IS457-fin-v1.json")
viz_v1

#change mean of score scaling 


/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib

alt.HConcatChart(...)

#change mean of score scaling 
#no dots on secondary graph


In [13]:
clicker = alt.selection_point(fields=['Franchise name'])
hover = alt.selection_point(on='mouseover', nearest=True)

base1 = alt.Chart(source).properties(
    width=750,
    height=350
)

bar1 = base1.mark_bar(size=25).encode(
    x=alt.X("mean(score):Q"),
    y=alt.Y("Franchise name:N").sort('-x'),
    color=alt.condition(hover,alt.value('orange'),alt.value('#1f77b4')),
    tooltip=[alt.Tooltip('mean(score):Q', title='Average Bechdel Test Score'), 
    alt.Tooltip('count(Franchise name):Q', title='Number of films in franchise'),
    alt.Tooltip('Date:T', aggregate='min', title='First Film in Franchise Release Date'),
    alt.Tooltip('Date:T', aggregate='max', title='Most Recent Film in Franchise Release Date')],
).add_params(clicker, hover).transform_filter(clicker).properties(
    width=750,
    height=350
)

xrule = (
    base1.mark_rule(strokeDash=[12, 6], size=2).encode(x=alt.datum(df1["score"].mean()))
)

viz1_v1 = bar1 + xrule
viz1_v1.properties().save("IS457-fin-viz1.json")
viz1_v1 


/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib

alt.LayerChart(...)

In [14]:
selector = alt.selection_point(fields=['Franchise name'])

base2 = alt.Chart(source).properties(
    width=500,
    height=500
)

bars = base2.mark_bar(size=30).encode(
    x=alt.X("mean(score):Q"),
    y=alt.Y("Franchise name:N").sort('-x'),
    #color=alt.condition(selector,'Franchise name:N',alt.value('lightgray'),scale=color_scale),
    tooltip=[
        alt.Tooltip('mean(score):Q', title='Average Bechdel Test Score'), 
        alt.Tooltip('count(Franchise name):Q', title='Number of films in franchise'), 
        alt.Tooltip('Date:T', aggregate='min', title='First Film in Franchise Release Date'),
        alt.Tooltip('Date:T', aggregate='max', title='Most Recent Film in Franchise Release Date')],
).add_params(selector).transform_filter(selector)

xrule = (
    base2.mark_rule(strokeDash=[12, 6], size=2).encode(x=alt.datum(df1["score"].mean()))
)

lines = base2.mark_line(point=True).encode(
    x=alt.X('Date:T'), #alt.condition(selector,'symbol:N',alt.value('lightgray'),scale=color_scale),
    y=alt.Y('score:Q'),
    color=alt.condition(selector,alt.value('orange'),alt.value('lightgray')),
    tooltip=[alt.Tooltip('Movie name:N', title='Film: '), alt.Tooltip('Date:T', title='Date released in USA'), alt.Tooltip('score:Q', title='Test Passed'),],
).transform_filter(selector)

yrule = (
    base2.mark_rule(strokeDash=[12, 6], size=2).encode(y=alt.datum(df1["score"].mean()))
)

dep = lines + yrule
prime = bars + xrule

viz_v1 = alt.hconcat(prime, dep).configure_point(size=120)

viz_v1.properties().save("IS457-fin-viz2.json")
viz_v1

/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib

alt.HConcatChart(...)